In [7]:
pip install gradio openai pytesseract PyPDF2 pdf2image pillow

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install fpdf2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement deepseek-api (from versions: none)
ERROR: No matching distribution found for deepseek-api
Note: you may need to restart the kernel to use updated packages.


In [29]:
import gradio as gr
import openai
from PyPDF2 import PdfReader
from PIL import Image
import pytesseract
import tempfile
import os
from docx import Document
from fpdf import FPDF
from dotenv import load_dotenv
from openai import OpenAI

ModuleNotFoundError: No module named 'deepseek'

In [31]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if deepseek_api_key:
    print(f"Deepseek API Key exists and begins {deepseek_api_key[:7]}")
else:
    print("Deepseek API Key not set")


openai = OpenAI()

deepseek = OpenAI(
    base_url="https://api.deepseek.com/v1",  # Verify endpoint
    api_key=deepseek_api_key
)

OpenAI API Key exists and begins sk-proj-
Deepseek API Key exists and begins sk-815b


In [12]:
def extract_text_from_pdf(pdf_file):
    """
    Extracts text from PDF files using PyPDF2 and OCR for image-based PDFs.
    """
    text = ""
    with tempfile.TemporaryDirectory() as temp_dir:
        # Use the file path directly
        pdf_path = pdf_file.name

        # Read PDF with PyPDF2
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            if page.extract_text():
                text += page.extract_text()
            else:
                # If no text is found, use OCR
                images = convert_pdf_to_images(pdf_path)
                for img in images:
                    text += pytesseract.image_to_string(img)

    return text.strip()


In [13]:
def convert_pdf_to_images(pdf_path):
    """
    Converts each page of a PDF into an image using PIL.
    """
    from pdf2image import convert_from_path
    images = convert_from_path(pdf_path)
    return images

In [14]:
def parse_resume_with_openai(resume_text):
    """
    Sends the extracted resume text to OpenAI for parsing and summarization.
    """
    prompt = (
        "You are an AI assistant specialized in analyzing resumes. Please summarize the following resume, "
        "extract key details such as name, contact information, skills, experiences, and education:\n\n"
        f"{resume_text}\n\n"
        "Provide the summary in JSON format with the keys: name, contact, skills, experiences, education."
    )

    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an AI assistant for resume parsing."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [15]:
def generate_cover_letter_with_openai(parsed_resume_json, company_name, job_description, notes, tone, name, phone, email):
    """
    Uses OpenAI to generate a cover letter based on the parsed resume JSON, company name, and job description.
    """
    prompt = (
        "You are an AI assistant specialized in writing cover letters. Based on the following parsed resume JSON, "
        "create a professional cover letter that highlights the candidate's skills and experiences, and "
        "explains why they are a great fit for the following job opportunity:\n\n"
        f"Parsed Resume: {parsed_resume_json}"
        f"Name: {name}"
        f"Phone: {phone}"
        f"Email: {email}"
        f"Company Name: {company_name}\n"
        f"Job Description: {job_description}"
        f"Notes: {notes}"
        f"Tone: {tone}"
        "Dont include placeholder item such as : [Date] [Company Name] [Company Address] in cover letter generated"
        "Dont include placeholder item such as : [Date] [Company Name] [Company Address] in cover letter generated"
        "Dont include placeholder item such as : [Date] [Company Name] [Company Address] in cover letter generated"
        "The cover letter should be well-structured using user name, email, and phone number as header and tailored to the company and role."
    )

    system_prompt = """
    You are an AI assistant for writing professional cover letters. Your responses must be enthusiastic, confident, and positive.\n
    Dont include placeholder item such as : [Date] [Company Name] [Company Address] in cover letter generated
    """
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [32]:
def generate_cover_letter_with_deepseek(parsed_resume_json, company_name, job_description, notes, tone, name, phone, email):
    """
    Uses DeepSeek to generate a cover letter based on the parsed resume JSON, company name, and job description.
    """
    prompt = (
        "You are an AI assistant specialized in writing cover letters. Based on the following parsed resume JSON, "
        "create a professional cover letter that highlights the candidate's skills and experiences, and "
        "explains why they are a great fit for the following job opportunity:\n\n"
        f"Parsed Resume: {parsed_resume_json}"
        f"Name: {name}"
        f"Phone: {phone}"
        f"Email: {email}"
        f"Company Name: {company_name}\n"
        f"Job Description: {job_description}"
        f"Notes: {notes}"
        f"Tone: {tone}"
        "Do not include placeholder items such as: [Date], [Company Name], or [Company Address] in the generated cover letter. "
        "The cover letter should be well-structured using the user's name, email, and phone number as the header, "
        "and tailored to the company and role."
    )

    system_prompt = """
    You are an AI assistant for writing professional cover letters. Your responses must be enthusiastic, confident, and positive.
    Never include placeholder items like [Date], [Company Name], or [Company Address] in the generated content.
    """

    response = deepseek.chat.completions.create(
        model="deepseek-chat",  # Check latest model name in DeepSeek docs
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=1000
    )
    
    return response.choices[0].message.content

In [24]:
def process_resume_and_cover_letter(pdf_file, company_name, job_description, notes, tone, name, phone, email):
    """
    Combines all steps: Extract text from PDF, parse using OpenAI, generate a cover letter, and return the cover letter.
    """
    extracted_text = extract_text_from_pdf(pdf_file)
    parsed_resume = parse_resume_with_openai(extracted_text)
    cover_letter = generate_cover_letter_with_deepseek(parsed_resume, company_name, job_description, notes, tone, name, phone, email)
    return cover_letter

In [34]:
def save_cover_letter_as_docx(cover_letter, name, company_name):
    """
    Save the cover letter as a .docx file.
    """
    file_name = f"Cover Letter {name} for {company_name}.docx"
    file_path = os.path.join(tempfile.gettempdir(), file_name)
    doc = Document()
    doc.add_paragraph(cover_letter)
    doc.save(file_path)
    return file_path

def save_cover_letter_as_pdf(cover_letter, name, company_name):
    """
    Save the cover letter as a .pdf file using DejaVu Sans for Unicode compatibility.
    """
    file_name = f"Cover Letter {name} for {company_name}.pdf"
    file_path = os.path.join(tempfile.gettempdir(), file_name)

    # Use current working directory for the font path
    base_dir = os.getcwd()  # Directory of the current working environment
    font_path = os.path.join(base_dir, "../fonts/DejaVuSans.ttf")

    # Ensure the font file exists
    if not os.path.exists(font_path):
        raise RuntimeError(f"Font file not found: {font_path}")

    # Initialize FPDF
    pdf = FPDF(format="A4", unit="mm")
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    # Add and set the Unicode-compatible font
    pdf.add_font("DejaVu", "", font_path)
    pdf.set_font("DejaVu", size=12)

    # Add the cover letter content
    for paragraph in cover_letter.split("\n\n"):
        pdf.multi_cell(0, 10, text=paragraph, border=0, align="L")
        pdf.ln(5)  # Add spacing between paragraphs

    pdf.output(file_path)
    return file_path

In [35]:
# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Cover Letter Generator with Download Options")

    with gr.Row():
        with gr.Column():
            pdf_input = gr.File(label="Upload PDF Resume", file_types=[".pdf"])
            name_input = gr.Textbox(label="Full Name", placeholder="Enter your full name", lines=1)
            phone_input = gr.Textbox(label="Phone Number", placeholder="Enter your phone number", lines=1)
            email_input = gr.Textbox(label="Email Address", placeholder="Enter your email address", lines=1)
            company_name_input = gr.Textbox(label="Company Name", placeholder="Enter the company name", lines=1)
            job_description_input = gr.Textbox(label="Job Description", placeholder="Enter the job description", lines=5)
            notes_input = gr.Textbox(label="Notes (Optional)", placeholder="Highlight specific skills or expertise", lines=3)
            tone_input = gr.Dropdown(label="Tone", choices=["Super Formal", "Semi-Formal", "Normal (Formal)", "Humble"], value="Normal (Formal)")

        with gr.Column():
            cover_letter_output = gr.Textbox(label="Generated Cover Letter", lines=15)
            docx_download_btn = gr.File(label="Download as DOCX")
            pdf_download_btn = gr.File(label="Download as PDF")

    
    # Function to handle cover letter generation and file creation
    def generate_and_save_cover_letter(pdf_file, company_name, job_description, notes, tone, name, phone, email):
        cover_letter = process_resume_and_cover_letter(pdf_file, company_name, job_description, notes, tone, name, phone, email)
        docx_path = save_cover_letter_as_docx(cover_letter, name, company_name)
        pdf_path = save_cover_letter_as_pdf(cover_letter, name, company_name)
        return cover_letter, docx_path, pdf_path

    # Button click to generate cover letter and enable downloads
    submit_btn = gr.Button("Generate Cover Letter")
    submit_btn.click(
        generate_and_save_cover_letter,
        inputs=[pdf_input, company_name_input, job_description_input, notes_input, tone_input, name_input, phone_input, email_input],
        outputs=[cover_letter_output, docx_download_btn, pdf_download_btn]
    )

# Launch the Gradio app
demo.launch(share = True)

* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://fde1eb0537c515f2ac.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
